# AgIT Thingsboard Export Notebook

## Import Modules

In [1]:
import requests
import sys
import os
import json
import pandas as pd
from pprint import pprint
import datetime
import pytz
import config

deviceList = []

## Set Variables

In [2]:
config = {
 'username' : 'EMAILADDRESS@purdue.edu', ### Insert your email address used by AgIT Thingsboard system
 'password': 'PASSWORD',  ### Insert your AgIT thingsboard password
 'server' : 'https://things.iot.ag.purdue.edu:8080'
}

## Define Functions

In [3]:
def getCustomerDevices(custID, textSearch=None):
    parameters = {        
        'pageSize': 1000,
        'page': 0,                
    }
    att_parms = {
        'keys': 'dev_eui'
    }
    if(textSearch):
        parameters.update({'textSearch': textSearch})
    responseList = requests.get(f"{config['server']}/api/customer/{custID}/devices", headers=TBheaders,params= parameters).json()
    #pprint(responseList)
    list = []
    for dev in responseList['data']:
        #pprint(dev)
        #print('------------------------------------------------------------------------------------------')
        #'id': {'entityType': 'DEVICE', 'id': 'd49153a0-c868-11eb-95d8-09d06ef6a9a5'},
        url = f"{config['server']}/api/plugins/telemetry/DEVICE/{dev['id']['id']}/values/attributes"
        deviceResp = requests.get(url, headers=TBheaders,params= att_parms).json()
        #print('------------------------------------------------------------------------------------------')
        list.append([dev['id']['id'],dev['name'],deviceResp[0]['value']])
    return list
        

def login(url, username, password):
    # Log into ThingsBoard
    return requests.post(f"{url}/api/auth/login", json={
        "username": username,
        "password": password
    }).json()['token']

def get_keys(device):
    return requests.get(f"{config['server']}/api/plugins/telemetry/DEVICE/{device}/keys/timeseries",
                 headers=TBheaders).json()
def get_data_chunk(url, token, device, key, start, stop, limit):
    #print([url, device, key, start, stop, limit])
    return requests.get(f"{url}/api/plugins/telemetry/DEVICE/{device}/values/timeseries",
             headers=TBheaders,
            params= {
                'keys': key,
                'startTs': start,
                'endTs': stop,
                'limit': limit,
                'agg': 'NONE'
            }).json()

def get_data(url, token, device, key, start, stop):
    global totalLength
    p = pd.DataFrame()
    
    # You have to request data backwards in time ...
    while start < stop:
        data = get_data_chunk(url, token, device[0], key, start, stop, 100000)
        #print(data)
        if key not in data:
            break;
        
        #print(f"{key}: Loaded {len(data[key])} points")
        t = pd.DataFrame.from_records(data[key])
        #t['Timestamp'] = t['ts']
        #pprint(t['ts'])
        t['ts'] = (pd.to_datetime(t['ts'],unit='ms'))        
        t.set_index('ts', inplace=True)
        
        t.rename(columns={'value': key}, inplace=True)
        p = p._append(t)

        # Update "new" stop time
        stop = data[key][-1]['ts'] - 1
    totalLength += len(p)
    #print(f"Total Length: {totalLength}")
    return p

def outputCSV(devices):
    global totalLength
    final_df = pd.DataFrame()
    for device in devices:
        #print(f"Downloading DEVICE: {device[0]} data");
        #print(device)
        p = pd.DataFrame()
        for key in keys:
            #print(f"info: Pulling {key}...");
            tempin = get_data(config['server'], token, device, key, startTS, endTS)            
            if(len(tempin)>0):                
                p = pd.concat([p,tempin], axis=1)
        p['Entity Name'] = device[1]
        p['dev_eui'] = device[2]
        p.reset_index(drop=False)
        #p_new_index = p.assign(**{'Timestamp': p.index})        
        if(len(p)):
            final_df = pd.concat([final_df,p])
        
    # Create Time Strings
    # Convert to nanoseconds for pandas.to_datetime
    start_timestamp_ns = startTS * 1000000
    end_timestamp_ns = endTS * 1000000
    
    # Convert timestamp to datetime object
    start_dt = pd.to_datetime(start_timestamp_ns, unit='ns')
    end_dt = pd.to_datetime(end_timestamp_ns, unit='ns')
    
    # Format datetime string as yyyy-mm-dd-HH-MM
    start_formatted_string = start_dt.strftime('%Y-%m-%d-%H-%M')
    end_formatted_string = end_dt.strftime('%Y-%m-%d-%H-%M')
    df_order = ["Entity Name","data_soil_moisture1","data_soil_moisture2","data_soil_moisture3","data_soil_moisture4","data_tem1","data_tem2","data_tem3","data_tem4","data_tem5","data_tem6","data_tem7","dev_eui"]
    final_df = final_df.reindex(columns=df_order)
    final_df1 = final_df.sort_values(by='ts')
    
    # Get current time
    now = datetime.datetime.now()
    
    # Format time string (hours and minutes)
    formatted_time = now.strftime("%H-%M")
    final_df1.to_csv(f"./data-{start_formatted_string}_{end_formatted_string}_{formatted_time}.csv")
    print("Done.")

def getDeviceCredentialsByDeviceId(deviceID = 0):
    url = config['server']+'/device/'+deviceID+'/credentials'
    resp = requests.get(url,headers=TBheaders)
    responseList = resp.json()
    #pprint(responseList)
    return responseList['credentialsID']

def getDeviceServerAttributes(deviceID = 0):
    if deviceID == 0:
        while(deviceID == 0):
            try:
                deviceID = input("Enter device ID: ")
            except:
                print("Invalid DeviceID")
    url = config['server']+'/plugins/telemetry/DEVICE/'+deviceID+'/values/attributes'
    #pprint(url)
    #pprint(TBheaders)
    xresp = requests.get(url,headers=TBheaders)
    #pprint(xresp)
    #pprint(resp.content())
    #print(xresp.text())
    responseList = xresp.json()
    #pprint(responseList)
    #return responseList['credentialsID']



## Get Token

In [5]:
print("Server: ",config['server'])
token = login(config['server'], config['username'], config['password']);
print(f"Token: {token}")
TBheaders={ 'Accept': '*/*', 'X-Authorization': f"Bearer {token}" }

Server:  https://things.iot.ag.purdue.edu:8080


KeyError: 'token'

## Set Timeframe for data pull

In [ ]:
# Create a datetime object representing the local date and time
# Year, Month, Day, Hour, Minute
start_dt = datetime.datetime(2023, 10, 15, 0, 0)
end_dt = datetime.datetime(2023, 10, 20, 23, 59)

# Convert to a specific time zone (e.g., UTC)
start_tz_utc = pytz.timezone("UTC")
start_dt_utc = start_tz_utc.localize(start_dt)
end_tz_utc = pytz.timezone("UTC")
end_dt_utc = end_tz_utc.localize(end_dt)

# Extract the Unix timestamp
startTS = int(start_dt_utc.timestamp())*1000
endTS = int(end_dt_utc.timestamp())*1000

# Use for relative time frames
#startTS = int((datetime.now() - timedelta(days=30)  - datetime(1970, 1, 1)).total_seconds() * 1000) # 30 days ago
#endTS = int((datetime.datetime.utcnow() - datetime.datetime(1970, 1, 1)).total_seconds() * 1000) # now

print(startTS, endTS)

## Get list of devices by "Customer"

In [ ]:
# getCustomerDevices(custID, textSearch=None):
# 7576b020-ecae-11ec-b72b-5dd76ca52a2b = Cherkhauer Customer ID
# ABE-DRAGINO-GROPOINT-CHERKHAUER = Devices with names beginning with "ABE-DRAGINO-GROPOINT-CHERKHAUER"
devices = getCustomerDevices("7576b020-ecae-11ec-b72b-5dd76ca52a2b","ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE")
pprint(devices)

## Set keys to retrieve

In [ ]:
totalLength = 0
# keys to retrieve
#keys = ["data_TempC_SHT","data_Hum_SHT"]
#keys = ["data_ambient_temperature","data_input1_frequency","data_input1_frequency_to_moisture","data_Input2_voltage","data_Input2_voltage_to_temp","data_light_intensity","data_relative_humidity"]
keys = ["data_soil_moisture1","data_soil_moisture2","data_soil_moisture3","data_soil_moisture4","data_tem1","data_tem2","data_tem3","data_tem4","data_tem5","data_tem6","data_tem7"]


## Run Function to output data

In [ ]:
outputCSV(devices)